# 01 — Data Collection

Downloads input ticker (Adj Close, Volume) and VIX (Close) from Yahoo Finance with smart caching.
Saves combined raw data for the next stage.

In [1]:
import sys
import os
from pathlib import Path

# ── Project configuration ──────────────────────────────────────────────────────
PROJECT_FOLDER = "spy_run"   # Change to "default_run" for full training

# ── Set up paths ───────────────────────────────────────────────────────────────
PROJECT_ROOT = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_ROOT))

DATA_CONFIG = PROJECT_ROOT / "config" / PROJECT_FOLDER / "data_config.json"
MODEL_CONFIG = PROJECT_ROOT / "config" / PROJECT_FOLDER / "model_config.json"

print(f"Project root : {PROJECT_ROOT}")
print(f"Project folder: {PROJECT_FOLDER}")
print(f"Data config  : {DATA_CONFIG}")

Project root : /Users/sharannaribole/Documents/github/stock-return-classifier
Project folder: spy_run
Data config  : /Users/sharannaribole/Documents/github/stock-return-classifier/config/spy_run/data_config.json


In [2]:
from src.utils.config_loader import ConfigLoader

config = ConfigLoader(str(DATA_CONFIG), str(MODEL_CONFIG))
print(config)
print()
print(f"Ticker     : {config.get('ticker')}")
print(f"Start date : {config.get('start_date')}")
print(f"End date   : {config.get('end_date')}")

ConfigLoader(ticker=SPY, project=spy_run)

Ticker     : SPY
Start date : 2006-01-01
End date   : 2026-02-17


## Collect Data

In [3]:
import os
os.chdir(PROJECT_ROOT)  # ensure relative paths work

from src.data.collector import DataCollector

collector = DataCollector(config)
raw_df = collector.collect_data()
raw_df.head()

  Saved to cache: SPY_20051009_20260217.csv
  Saved to cache: VIX_20051009_20260217.csv
Combined data: 5119 rows, 5 columns


,Adj_Close,High,Low,Volume,VIX_Close
Date,,,,,
2005-10-10,81.439987,82.202200,81.233987,52677000,15.550000
2005-10-11,81.323219,81.982429,81.247684,75629800,15.630000
2005-10-12,80.684639,81.803922,80.622840,100510400,16.219999
2005-10-13,80.636581,81.082923,80.258907,99052900,16.469999
2005-10-14,81.488014,81.584149,80.725802,88651000,14.870000


## Data Summary

In [4]:
import pandas as pd

print(f"Date range : {raw_df.index[0].date()} to {raw_df.index[-1].date()}")
print(f"Total rows : {len(raw_df)}")
print(f"Columns    : {list(raw_df.columns)}")
print()
print(f"Missing values:")
print(raw_df.isnull().sum())
print()
print(f"Data types:")
print(raw_df.dtypes)

Date range : 2005-10-10 to 2026-02-13
Total rows : 5119
Columns    : ['Adj_Close', 'High', 'Low', 'Volume', 'VIX_Close']

Missing values:
Adj_Close    0
High         0
Low          0
Volume       0
VIX_Close    0
dtype: int64

Data types:
Adj_Close    float64
High         float64
Low          float64
Volume         int64
VIX_Close    float64
dtype: object


In [5]:
raw_df.describe()

,Adj_Close,High,Low,Volume,VIX_Close
count,5119.000000,5119.000000,5119.000000,5.119000e+03,5119.000000
mean,233.729393,234.986188,232.285224,1.252423e+08,19.370766
std,160.502399,161.254409,159.607970,9.069525e+07,8.696475
min,49.944592,51.330515,49.203951,1.527000e+07,9.140000
25%,101.015812,101.451454,100.390037,6.657780e+07,13.650000
50%,175.186035,175.880154,174.312177,9.471770e+07,16.990000
75%,346.946243,350.875382,343.744533,1.537868e+08,22.360001
max,695.489990,697.840027,693.940002,8.710263e+08,82.690002


## Save Raw Data

In [6]:
raw_dir = PROJECT_ROOT / "data" / "raw"
raw_dir.mkdir(parents=True, exist_ok=True)

out_path = raw_dir / f"{PROJECT_FOLDER}_raw.parquet"
raw_df.to_parquet(out_path)
print(f"Saved raw data to: {out_path}")
print(f"File size: {out_path.stat().st_size / 1024:.1f} KB")

Saved raw data to: /Users/sharannaribole/Documents/github/stock-return-classifier/data/raw/spy_run_raw.parquet
File size: 239.2 KB
